# Solar Panel Detection

This notebook demonstrates how to use the geoai package for solar panel detection using a pre-trained model.

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/solar_panel_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
%pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7

## Import libraries

In [16]:
import geoai
import requests

url = 'https://drive.google.com/uc?export=download&id=1QYEvDsiUyWYNRHk7rvoYdUwXl7EduzOA'
response = requests.get(url)

with open('downloaded_image.tif', 'wb') as f:
    f.write(response.content)

## Download sample data

In [14]:
# raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/solar_panels_davis_ca.tif"
raster_path = geoai.download_file("https://drive.google.com/file/d/1QYEvDsiUyWYNRHk7rvoYdUwXl7EduzOA/view?usp=sharing")

File already exists: view?usp=sharing


In [15]:
geoai.print_raster_info(raster_path)

Error reading raster: 'view?usp=sharing' not recognized as being in a supported file format.


## Visualize data

In [5]:
geoai.view_raster(raster_url)

## Initialize model

In [6]:
detector = geoai.SolarPanelDetector()

Model path not specified, downloading from Hugging Face...


solar_panel_detection.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/75788bf6253e967a91b9576b1620949a3b939820/solar_panel_detection.pth
Model loaded successfully


## Generate masks

In [ ]:
output_path = "solar_panel_masks.tif"

In [ ]:
masks_path = detector.generate_masks(
    raster_path,
    output_path=output_path,
    confidence_threshold=0.4,
    mask_threshold=0.5,
    min_object_area=100,
    overlap=0.25,
    chip_size=(400, 400),
    batch_size=4,
    verbose=False,
)

## Visualize masks

In [ ]:
geoai.view_raster(
    output_path,
    indexes=[2],
    colormap="autumn",
    layer_name="Solar Panels",
    basemap=raster_url,
)

## Vectorize masks

In [ ]:
gdf = geoai.orthogonalize(
    input_path=masks_path, output_path="solar_panel_masks.geojson", epsilon=0.2
)

## Visualize initial results

In [ ]:
geoai.view_vector_interactive(gdf, tiles=raster_url)

## Calculate geometric properties

In [ ]:
gdf = geoai.add_geometric_properties(gdf)
gdf.head()

In [ ]:
print(len(gdf))

In [ ]:
geoai.view_vector_interactive(gdf, column="elongation", tiles=raster_url)

## Filter results

In [ ]:
gdf_filter = gdf[(gdf["elongation"] < 10) & (gdf["area_m2"] > 5)]
print(len(gdf_filter))

## Visualize final results

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="area_m2", tiles=raster_url)

In [ ]:
geoai.view_vector_interactive(
    gdf_filter, style_kwds={"color": "red", "fillOpacity": 0}, tiles=raster_url
)

In [ ]:
gdf_filter["area_m2"].hist()

In [ ]:
gdf_filter["area_m2"].describe()

In [ ]:
gdf_filter["area_m2"].sum()

## Save results

In [ ]:
gdf_filter.to_file("solar_panels.geojson")

![image](https://github.com/user-attachments/assets/a38925dc-b840-42b0-a926-326ef99b181c)